## Treść zadania

W grze bierze udział 𝑟 graczy, przy czym *i*-ty gracz posiada *n<sub>i</sub>* żetonów,
*n<sub>i</sub>* > 0, *i* = 1, ... , *r*. W każdej turze wybieranych jest losowo dwóch graczy, a zwycięzca zabiera jeden żeton przegranemu. Gracz, który zostanie bez żetonów jest eliminowany z gry, która jest kontynuowana,
dopóki jeden gracz nie zbierze wszystkich *n* żetonów. Ten gracz wygrywa całą grę.

Zakładając, że wyniki rozgrywek w kolejnych turach są od siebie niezależne, a w każdej z nich szanse na wygraną każdego z dwóch graczy są jednakowe (1/2), znajdź *P<sub>i</sub>* – prawdopodobieństwo, że całą grę
wygra gracz *i*-ty.

W celu rozwiązania zadania przeprowadź symulację komputerową. Eksperymentuj z różnymi początkowymi układami żetonów, dla każdego układu przeprowadź wystarczająco dużo symulacji, aby otrzymać wiarygodne wyniki (np. 10,000 symulacji). Czy dostrzegasz jakiś związek między wartościami *n<sub>i</sub>* i *n* a *P*?

In [1]:
import random as r
from datetime import datetime as dt
from operator import itemgetter as ig

In [2]:
class Player:
    def __init__(self, id_num: int, coins: int):
        self.id = id_num
        self.coins = coins
        self.starting_coins = coins
        
        
    def can_play(self) -> bool:
        return self.coins > 0
    
    
    def remove_coin(self):
        self.coins -= 1
    
    
    def add_coin(self):
        self.coins += 1

In [3]:
class Game:
    def __init__(self, players_num: int, coins_num: int):
        self.players_num = players_num
        self.coins_num = coins_num
        self.rounds_num = 0
        self.playing_players = []
        self.lost_players = []
        self.players_prob = [0.0 for _ in range(self.players_num)]
        self.start_time = dt.now()
        
        
    def shuffle_coins(self):
        self.shuffled_coins = [1 for _ in range(self.players_num)]
        
        for _ in range(self.coins_num - self.players_num):
            index = r.randrange(self.players_num)
            self.shuffled_coins[index] += 1
    
    
    def reset(self):
        self.playing_players = []
        self.lost_players = []

        for id_num, coins in enumerate(self.shuffled_coins):
            self.playing_players.append(Player(id_num, coins))
    
    
    def play(self):
        self.rounds_num += 1
        while len(self.playing_players) > 1:
            players = r.sample(self.playing_players, k=2)
            self.__players_play(players)
        
        self.players_prob[self.playing_players[0].id] += 1
        # self.print_data()
    
    def __players_play(self, players):
        players[0].add_coin()
        players[1].remove_coin()

        if not players[1].can_play():
            self.playing_players.remove(players[1])
            self.lost_players.append(players[1])
            
    def print_results(self):
        self.players_prob = [x/self.rounds_num for x in self.players_prob]

        self.lost_players.append(self.playing_players[0])
        
        out = [(x.id, x.starting_coins, self.players_prob[x.id]) for x in self.lost_players]
        
        print(f'Players: {self.players_num}, coins: {self.coins_num}, took: {(dt.now() - self.start_time).total_seconds()} seconds')
        print('id \t starting_coins \t winning_prob')
        for player in sorted(out, key=ig(2), reverse=True):
            print(f'{player[0]} \t {player[1]} \t\t\t {player[2]}')
        
    
    def print_data(self):
        print(self.shuffled_coins)
        print([x.id for x in self.playing_players])
        print([x.id for x in self.lost_players])
        print(end='\n\n')

In [4]:
# CONSTANTS
PLAYERS = 15
COINS = 100
ROUNDS = 10_000

In [9]:
for c in range(COINS, COINS*9, COINS*2):      # 100 -> 1000
    game = Game(PLAYERS, c)

    game.shuffle_coins()

    for _ in range(ROUNDS):
        game.reset()
        game.play()

    game.print_results()

[0, 1, 2, 3, 4]


KeyboardInterrupt: 

## Konkluzje

Z przeprowadzonych symulacji można wysnuć wniosek, że gracze, którzy na początku dostawali większą liczbę monet, radzili sobie zdecydowanie lepiej, niż gracze z ich mniejszą liczbą. Zarazem, gracze z największą liczbą monet wygrywali istotnie więcej razy niż reszta.